__[Open and try this file online (Colab)](https://colab.research.google.com/github/djekra/wiktionary_wortschatz/blob/master/1_wiktionary_parsen/x28_Templates_überprüfen.ipynb)__

# Templates überprüfen
* dient nur zur Kontrolle, ist nicht notwendig zur Erstellung des Wörterbuches
* erstellt die Datei wiktionary_templates.pickle
* parst dafür wikitext mit mwparserfromhell. 

Dauer: Auf Colab 45 min

## Import

In [1]:
# blab init
try:
    import blab
except ImportError as e:
    !pip install blab
    import blab    
startup_notebook = blab.blab_startup()
%run $startup_notebook 

blab init
environment['in_colab']     = False
environment['dropbox_path'] = /home/me/Data_Linux/Dropbox
environment['lib_path']     = /home/me/Data_Linux/Dropbox/31_Projekte/01_Python/libs
Start Time: 14:22:31


In [2]:
# pandas
if environment['in_colab']:
    !pip install --upgrade pandas
import pandas as pd

In [2]:
# alle anderen
try:
    import pandasklar as pak, bj_nlp, mwparserfromhell, swifter, bpyth as bpy

except ImportError as e:
    !pip install pandasklar bj_nlp mwparserfromhell swifter
    import pandasklar as pak, bj_nlp, mwparserfromhell, swifter, bpyth as bpy
    
grid       = pak.grid
check_mask = pak.check_mask      

time: 5.94 s


## Einstellungen

In [4]:
# Einstellungen

version = '_all'
#version = '_short'

# Load
verzeichnis_load                      = 'data_00'
wiktionary_filename                   = verzeichnis_load + '/wiktionary'              + version + '.pickle'  
wiktionary_merkmal_filename           = verzeichnis_load + '/wiktionary_merkmal'      + version + '.pickle'  
wiktionary_merkmal_text_filename      = verzeichnis_load + '/wiktionary_merkmal_text' + version + '.pickle'  


# Save
verzeichnis_save                      = 'data_01'
wiktionary_templates_filename         = verzeichnis_save + '/wiktionary_templates'    + version + '.pickle'    

# Was debuggen?
suche_debug = ['Haus',]

## Vorbereiten

In [5]:
# Verzeichnisse erzeugen
os.makedirs(verzeichnis_load, exist_ok=True) 
os.makedirs(verzeichnis_save, exist_ok=True) 

In [6]:
# Steuertabellen und Python-Files von Github herunterladen
import os
if not os.path.exists('Steuertabellen'):
   !git clone https://github.com/djekra/wiktionary_wortschatz.git
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/Steuertabellen Steuertabellen
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/*.py .
#   !rm -r wiktionary_wortschatz

In [7]:
# Lade Ergebnisse der vorherigen Verarbeitungsstufe von Google Drive, falls nicht lokal vorhanden
# Dieses Verzeichnis wurde in der vorherigen Verarbeitungsstufe angelegt 
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av /content/drive/MyDrive/{verzeichnis_load} .

In [8]:
# Weitere Imports
from Steuertabellen.s01_Steuertabellen   import *

# mwparserfromhell plus eigene Ergänzungen 
import mwparserfromhell
from x24_process_wikicode                import *
from x22_my_node                         import *

## Funktionen

In [9]:
def parse_templatenames(zeile):
    '''
    Liefert alle verwendeten Templates eines Wiktionary-Abschnittes
    sowie die Länge der Parameter jedes Templates.
    '''
    
    # mwparserfromhell
    wikicode = mwparserfromhell.parse(zeile.wikitext)     
    
    templates = wikicode.filter_templates(recursive=False)  
    templatenames = list()
    for t in templates:  
        name = str(t.name.strip())
        lenp = str(len(t.params))
        if len(name) <= 2:
            continue
        if (len(name) == 3)  and  (name == str.lower(name)):
            continue            
        templatenames.append(name + '☉' + lenp)
    
    result = { 'section_id':    zeile.section_id,
               'section_id2':   zeile.section_id2,
               'templatenames': templatenames
             }

        
    return pd.DataFrame([result])

In [10]:
# häufigkeit             beschreibt, wie häufig ein Template insgesamt verwendet wird
# count                  beschreibt, wie oft ein Template pro Section verwendet wird.
# lenp_min und lenp_max  beschreiben die Anzahl der Parameter, die bei diesem Template im Mittel verwendet werden.
#                        Wenn ein Template pro Section mehrfach verwendet wird, können sich lenp_min und lenp_max unterscheiden.
#
def show_stats( wiktionary_templates, filter_templates, match_substring=False):
    '''
    Zeigt gruppierte Daten zur Verwendung der Templates im Wiktionary.
    * wiktionary_templates: die gleichnamige Tabelle
    * filter_templates:     Nur Templates, die in dieser Liste sind werden ausgewertet
    * match_substring:      Reicht in filter_templates der Anfang des Namens?
    
    Liefert folgende Daten für jedes Template:
    * lenp_min:     mittleres Minimum der Parameterlänge
    * lenp_max:     mittleres Maximum der Parameterlänge
    * count:        mittlere Häufigkeit pro Abschnitt  
    * section_id2:  Einige Beispiele, wo das Template vorkommt
    * häufigkeit:   Gesamthäufigkeit des Templates im ganzen Wiktionary
    '''
    filter_templates = tuple(filter_templates)
    spalten = ['section_id2','template','lenp_min', 'lenp_max', 'count' ]
    if match_substring:
        mask = wiktionary_templates.template.str.startswith(filter_templates)
    else:
        mask = wiktionary_templates.template.isin(filter_templates)
    df = pak.reset_index(wiktionary_templates[mask][spalten])
    
    gruppiert = df.groupby('template')
    
    result = gruppiert.agg(
      
        lenp_min    = ('lenp_min',    'mean'),   
        lenp_max    = ('lenp_max',    'mean'),  
        count       = ('count',       'mean'),            
        section_id2 = ('section_id2', pak.top_values_20),                 
        häufigkeit  = ('section_id2', 'count')                        

    ).reset_index()
 
    return result.sort_values('häufigkeit', ascending=False)

## Laden

In [12]:
# Laden
wiktionary = pak.load_pickle(wiktionary_filename)

In [13]:
try:
    wiktionary_templates = pak.load_pickle(wiktionary_templates_filename)  # Falls vorhanden
except:
    pass

In [14]:
#grid(wiktionary_templates,sample)

In [15]:
print('Einträge:', wiktionary.shape[0])

Einträge: 837593


In [16]:
mask = wiktionary.title.isin(suche_debug)
grid(wiktionary, mask)

2 rows out of 837593


In [ ]:
pak.memory_consumption(locals())

## parse_templatenames ausprobieren

In [18]:
blab.help(parse_templatenames)

<class 'IPython.core.display.Markdown'>


<span style="font-size:larger;">parse_templatenames(zeile):</span>

Liefert alle verwendeten Templates eines Wiktionary-Abschnittes
sowie die Länge der Parameter jedes Templates.

In [20]:
# Eine Testzeile aus wiktionary (als Series)
mask = wiktionary.lemma == 'Haus'
testzeile = wiktionary[mask].iloc[0]
testzeile

section_id                                                 21700
page_id                                                      217
page_part                                                      0
section_id2                                                 Haus
title                                                       Haus
lemma                                                       Haus
inflected                                                  False
pos                                           {'Substantiv': []}
rhymes                                                    [aʊ̯s]
section_len                                                70143
flexion        {'Genus': 'n', 'Nominativ Singular': 'Haus', '...
ipa                                                      [haʊ̯s]
syllables                                                 [Haus]
lang                                                     Deutsch
lang_code                                                     de
wikitext       === {{Wort

In [21]:
# parse_templatenames ausprobieren
df = parse_templatenames(testzeile)
df

,section_id,section_id2,templatenames
0,21700,Haus,"[Deutsch Substantiv Übersicht☉38, Worttrennung..."


In [22]:
df.iloc[0].templatenames[:10]

['Deutsch Substantiv Übersicht☉38',
 'Worttrennung☉0',
 'Pl.☉0',
 'Aussprache☉0',
 'IPA☉0',
 'Lautschrift☉1',
 'Hörbeispiele☉0',
 'Audio☉1',
 'Audio☉1',
 'Reime☉0']

## wiktionary_templates neu erstellen

In [23]:
# all: 15min
# short: 1min 30 sec
# apply erzeugt eine Series von DataFrames
# tolist macht daraus eine Liste
# und concat macht aus der Liste von DataFrames ein Gesamt-DataFrame
wiktionary_templates = pd.concat(wiktionary.swifter.apply(parse_templatenames, axis=1).tolist()  )

Pandas Apply:   0%|          | 0/837593 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# Alle Templates einzeln
wiktionary_templates = wiktionary_templates.explode('templatenames')

# split
wiktionary_templates[['template','lenp']] = wiktionary_templates['templatenames'].str.split('☉', n=1, expand=True)

In [ ]:
# lenp
wiktionary_templates.lenp = wiktionary_templates.lenp.astype(int)

In [ ]:
# Spalten pflegen
wiktionary_templates = pak.drop_cols(wiktionary_templates,['templatenames','section_id'])

In [ ]:
mask = wiktionary_templates.section_id2 == 'Haus'
wiktionary_templates[mask]

In [ ]:
# Manche Templates kommen mehrfach pro Abschnitt vor, daher wird gruppiert

# 7sec
gruppiert = wiktionary_templates.groupby(['section_id2','template'])

wiktionary_templates = gruppiert.agg(

    count      = ('lenp', 'count'),         
    lenp_min   = ('lenp', 'min'),  
    lenp_max   = ('lenp', 'max'),       
            
).reset_index()

In [ ]:
grid(wiktionary_templates, pak.sample)

In [ ]:
# Sparsame Datentypen
wiktionary_templates = pak.change_datatype(wiktionary_templates)
#pak.analyse_cols(wiktionary_templates)

## wiktionary_templates belöschen

In [ ]:
# Speicher freimachen
#if 'all' in version:
#    del wiktionary

In [ ]:
# Uninteressante Templates
templates0 =  ['Ü','K','Worttrennung','Aussprache','Sprache','Anmerkung','Reime'] 
templates0 += ['Übersetzungen','Ü-Tabelle','Referenzen','Quellen','IPA','Audio','Lautschrift','Hörbeispiele'] 
templates0 += ['ugs.','va.','landsch.','übertr.','abw.','trans.','intrans.','geh.','scherzh.','refl.','fachspr.','fam.']
templates0 += ['südd.','nordd.','österr.','schweiz.','vatd.','gmh.','goh.','lat.','ahd.','Üxx4','Hebr','CH&LI','fro.']
templates0 += ['mhd.','grc.','Polytonisch','Lautschrift?','Wort des Jahres']
templates0 += ['Sg.','Sg.1','Sg.2','Pl.','Pl.1','Pl.2','Pl.3','kPl.','Komp.','Sup.','Prät.','Part.','Gen.','Dat.','Akk.',]
templates0 += ['Beispiele fehlen','Literatur','Referenzen prüfen','Absatz']
templates0 += ['Wikipedia','Wikiquote','Wikipedia-Suche','Wikisource','Commons','Wikisource-Suche']
templates0 += ['Anmerkungen','Per-Deutsche Welle','Abschnitte fehlen','Dativ-e','Bekannte Namensträger','Fremdaffixe (Deutsch)']
templates0 += ['Internetquelle','Anmerkung scht','erweitern','überarbeiten','Wörter des Jahres']

In [ ]:
?show_stats

In [ ]:
# Uninteressante Templates
df = show_stats(wiktionary_templates, templates0)
grid(df)

In [ ]:
# belöschen
mask = wiktionary_templates.template.isin(templates0)
wiktionary_templates = pak.drop_rows(wiktionary_templates, mask)

In [ ]:
# reset_index
wiktionary_templates = pak.reset_index(wiktionary_templates)

In [ ]:
grid(wiktionary_templates, pak.sample)

## Speichern

In [ ]:
# Speichern
if wiktionary_templates_filename:
    pak.dump_pickle(wiktionary_templates, wiktionary_templates_filename) 

In [ ]:
# Save in Google Drive
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av {verzeichnis_save} /content/drive/MyDrive

In [ ]:
pak.memory_consumption(locals())

## Pläne überprüfen

In [ ]:
blab.render_doc(plan_merkmal_erstellen)

In [ ]:
# Steuertabelle
plan_merkmal = plan_merkmal_erstellen()
grid(plan_merkmal)

In [ ]:
# Alle Extraktionsmethoden
list(plan_merkmal.collect.unique())

In [ ]:
# Templates, deren Text eingesammelt wird
mask = (plan_merkmal.collect == 'T')
df = show_stats(wiktionary_templates, list(plan_merkmal[mask].template) )
grid(df)

In [ ]:
# Templates, deren Links eingesammelt werden
mask = (plan_merkmal.collect == 'L')
df = show_stats(wiktionary_templates, list(plan_merkmal[mask].template) )
grid(df)

In [ ]:
# Templates, deren Übersetzungen eingesammelt werden
mask = (plan_merkmal.collect == 'Ü')
df = show_stats(wiktionary_templates, list(plan_merkmal[mask].template) )
grid(df)

In [ ]:
# Templates, deren Parameter eingesammelt werden
mask = (plan_merkmal.collect == 'P')
df = show_stats(wiktionary_templates, list(plan_merkmal[mask].template) )
grid(df)

In [ ]:
# Templates, deren Existenz gecheckt wird
mask = (plan_merkmal.collect == 'E')
df = show_stats(wiktionary_templates, list(plan_merkmal[mask].template)).sort_values('häufigkeit', ascending=False)
grid(df)

## Templates finden, die noch ausgewertet werden könnten

In [ ]:
# Templates, die eine Gruppe mit gemeinsamen Oberbegriff kennzeichnen
templates1 =  ['Monate (Deutsch)','Wochentage (Deutsch)','Staaten Europa (Deutsch)','Verwandte (Deutsch)','Bundesstaaten der USA']
templates1 += ['Elemente','Periodensystem','Zahlensystem (Deutsch)','Planeten','Staaten Afrika (Deutsch)','Staaten Asien (Deutsch)']

# Templates, die noch nicht ausgelesen werden / werden können
templates2 =  ['Symbole','Reim'] # 'Wortart'
    
# Templates, die derzeit nicht interessante Informationen liefern
templates3 =  ['Entlehnungen']

known_templates = templates0 + templates1 + templates2 + templates3 + list(plan_merkmal.template)
#known_templates

In [ ]:
# show_stats (templates1 .. templates4 manuell durchschauen!)
df = show_stats(wiktionary_templates, templates3 )
grid(df)

In [ ]:
# restliche
df = pak.analyse_freqs( wiktionary_templates, 'template')
mask1 = ~df.template.isin(known_templates)   
mask2 = ~df.template.str.startswith(  ('Grundformverweis','Ref-','Lit-','QS','Ähnlichkeiten')  )
mask3 = ~df.template.str.endswith(    ('Übersicht','Übersicht m','Übersicht f','Übersicht -sch')  )
restliche = list(df[mask1  &  mask2  &  mask3 ].template)
#restliche

In [ ]:
# Kandidaten für Templates, die man noch auswerten könnte
# show_stats 
df = show_stats(wiktionary_templates, restliche ).sort_values('häufigkeit', ascending=False)
bgc('gold')
grid(df)

## Probeweises extract für ungenutzte Templates

In [ ]:
# Parameter
templatename     = 'Reim'
collectmethode   = 'P'

In [ ]:
# Section finden, welches ein bestimmtes Template enthalten
mask = wiktionary_templates.template == templatename
df = wiktionary_templates[mask].sort_values('count',ascending=False)
beispiellemma = df.iloc[1].section_id2
print('beispiellemma =', beispiellemma)
df.head(3)

In [ ]:
# plan für Testzwecke
plan = pak.dataframe({'template':templatename, 'collect':collectmethode,'merkmal':'TEST'})
plan

In [ ]:
mask = (wiktionary.section_id2 == beispiellemma)
testzeile = wiktionary[mask].iloc[0]
#testzeile

wikicode = mwparserfromhell.parse(testzeile.wikitext)   

df = extract_all(testzeile, plan, step=None )
df

## wiktionary_merkmal anschauen

In [ ]:
# Suche festlegen
suche_debug = ['Haus',]

In [ ]:
# Laden
wiktionary_merkmal      = pak.load_pickle(wiktionary_merkmal_filename)

In [ ]:
# wiktionary_merkmal
mask = wiktionary.title.isin(suche_debug)
suche_section_id2s = wiktionary[mask].section_id2.tolist()
mask = wiktionary_merkmal.section_id2.isin(suche_section_id2s)
a = pak.analyse_freqs( wiktionary_merkmal[mask], ['merkmal','data'], limits=[999,None])
grid(a)

In [ ]:
mask = (wiktionary_merkmal.node_kontext != '')
df = wiktionary_merkmal[mask]
a = pak.analyse_freqs( df, ['node_kontext','node_debug'], limits=[50,None] )
grid(a)

## wiktionary_merkmal_text anschauen

In [ ]:
# Laden
wiktionary_merkmal_text = pak.load_pickle(wiktionary_merkmal_text_filename)  # Falls vorhanden

In [ ]:
# wiktionary_merkmal_text
mask = wiktionary.title.isin(suche_debug)
suche_section_ids = wiktionary[mask].section_id.tolist()
mask = wiktionary_merkmal_text.section_id.isin(suche_section_ids)
grid(wiktionary_merkmal_text, mask)

In [ ]:
raise Stop